In [1]:
pip install pandas


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import requests
import pandas as pd
import json


In [3]:
# Substitua 'SUA_CHAVE_DE_API' pela sua chave de API
API_KEY = 'RGAPI-f996c09d-3987-4e3b-aaed-0fa0469d0c3b'

# Região do servidor (por exemplo, 'br1' para o servidor brasileiro)
REGION = 'br1'

# Nome de invocador (seu nome de jogador)
#SUMMONER_NAME = 'teteco25'

# URL base da API da Riot Games
BASE_URL = f'https://{REGION}.api.riotgames.com/lol'


In [4]:
listDivision = ['DIAMOND','EMERALD','PLATINUM','GOLD','SILVER','BRONZE','IRON']
listTier = ['I','II','III','IV']
#RANKED_SOLO_5x5

In [5]:
def filter_data(data):
    filtered_data = []
    for item in data:
        localized_names = item["localizedNames"].get("pt_BR")
        if localized_names:
            item["localizedNames"] = localized_names
            filtered_data.append(item)
    return filtered_data


In [6]:
def obter_id_invocador(SUMMONER_NAME):
    summoner_url = f'https://{REGION}.api.riotgames.com/lol/summoner/v4/summoners/by-name/{SUMMONER_NAME}'
    headers = {'X-Riot-Token': API_KEY}

    response = requests.get(summoner_url, headers=headers)

    if response.status_code == 200:
        match_history_data = response.json()
        return match_history_data['puuid']  # Retorna o ID do invocador
    else:
        print(f'Erro ao obter ID do invocador. Código de status: {response.status_code}')
        return None


In [7]:
def getConfigChallenge():
    summoner_url = f'https://{REGION}.api.riotgames.com/lol/challenges/v1/challenges/config'
    headers = {'X-Riot-Token': API_KEY}

    response = requests.get(summoner_url, headers=headers)

    if response.status_code == 200:
        filtered_data = filter_data(response.json())
        return filtered_data
    else:
        print(f'Erro ao obter ID do invocador. Código de status: {response.status_code}')
        return None

In [8]:
def getChallengeById(idPartida):
    match_history_url = f'https://{REGION}.api.riotgames.com/lol/challenges/v1/player-data/{idPartida}'
    headers = {'X-Riot-Token': API_KEY}

    response = requests.get(match_history_url, headers=headers)

    if response.status_code == 200:
        player_challenges_data = response.json()
        return player_challenges_data
    else:
        print(f'Erro ao obter histórico de partidas. Código de stasstus: {response.status_code}')
        return None

In [9]:
def getListNamesByElo(tier,division):
    list_player = f'https://{REGION}.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/{division}/{tier}?page=1'
    headers = {'X-Riot-Token': API_KEY}

    response = requests.get(list_player, headers=headers)

    if response.status_code == 200:
        player_challenges_data = response.json()
        return player_challenges_data
    else:
        print(f'Erro ao obter histórico de partidas. Código de stasstus: {response.status_code}')
        return None

In [10]:
if __name__ == '__main__':
    # Obter o ID do invocador
    account_id = obter_id_invocador('teteco25')

    if account_id:
        # Obter os dados dos desafios de configuração
        dfConfig = pd.DataFrame(getConfigChallenge())

        # Obter os dados do desafio do usuário
        dfUserChallenge = getChallengeById(account_id)
        
        df = pd.DataFrame(dfUserChallenge['challenges'])
        
        dfConfig['name'] = dfConfig['localizedNames'].apply(lambda x: x.get('name', ''))
        dfConfig['description'] = dfConfig['localizedNames'].apply(lambda x: x.get('description', ''))
        dfConfig['shortDescription'] = dfConfig['localizedNames'].apply(lambda x: x.get('shortDescription', ''))
        
        # Fundir os DataFrames df e dfConfig com base na correspondência entre df['challengeId'] e dfConfig['id']
        df_merged = pd.merge(df, dfConfig[['id', 'name','shortDescription']], left_on='challengeId', right_on='id', how='left')
        df_merged = df_merged.drop(columns=['id'])
        # Iterar sobre os desafios do usuário e imprimir o nome do desafio
        for index, row in df_merged.iterrows():
            print(f"ID: {row['challengeId']}")
            print(f"ID: {row['level']}")
            print(f"ID: {row['name']}")
            print(f"ID: {row['shortDescription']}")
            print("=" * 50)
    else:
        print("Nenhum histórico de partidas encontrado.")

ID: 0
ID: PLATINUM
ID: CRISTAL
ID: Regras especiais para o Cristal
ID: 1
ID: GOLD
ID: IMAGINAÇÃO
ID: Divisão IMAGINAÇÃO
ID: 2
ID: PLATINUM
ID: ESPECIALIZAÇÃO
ID: Divisão ESPECIALIZAÇÃO
ID: 3
ID: PLATINUM
ID: VIVÊNCIA
ID: Divisão VIVÊNCIA
ID: 4
ID: PLATINUM
ID: TRABALHO EM EQUIPE
ID: Divisão TRABALHO EM EQUIPE
ID: 5
ID: DIAMOND
ID: COLEÇÃO
ID: Divisão COLEÇÃO
ID: 302100
ID: DIAMOND
ID: Artimanha
ID: Ganhe pontos nos desafios da categoria Artimanha
ID: 302101
ID: MASTER
ID: Festinha no Nível 1
ID: Elimine Campões inimigos antes que os acampamentos da selva surjam
ID: 302102
ID: IRON
ID: Farme Campeões, Não Acampamentos
ID: Como caçador, consiga eliminações em rotas antes dos 10min
ID: 302103
ID: MASTER
ID: Presença Devastadora
ID: Jogando na rota, em 1 partida, consiga abates fora dela antes dos 10min
ID: 302104
ID: MASTER
ID: Rolezinho Nas Rotas
ID: Jogando na rota, consiga eliminações em todas as rotas antes dos 10min
ID: 302105
ID: GRANDMASTER
ID: Sistema Global de Defesa
ID: Tenha um

In [11]:
dfListNamesByElo = pd.DataFrame(getListNamesByElo(listTier[0],listDivision[0]))
dfResult = pd.DataFrame(columns=['summonerName', 'summonerID','tier','division'])

In [12]:
dfListNamesByElo_top5 = dfListNamesByElo.head(5)

for index, row in dfListNamesByElo_top5.iterrows():
    dfListNamesByElo_top5.loc[index, 'puuid'] = obter_id_invocador(row['summonerName'])


C:\Users\programador4\AppData\Local\Temp\ipykernel_4236\3022913596.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfListNamesByElo_top5.loc[index, 'puuid'] = obter_id_invocador(row['summonerName'])


In [13]:
dfListNamesByElo_top5

,leagueId,queueType,tier,rank,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak,puuid
0,4d26cbec-ef6b-4822-8f85-1ba0069e6b54,RANKED_SOLO_5x5,DIAMOND,I,0JFE711rwJFL1TI8xBdEoqYNUZDxIqL5N32bTiY2eNEglpk,OreoM,3,69,61,False,False,False,False,i0zml1uE_JcxxNljmHdw7nquqFmcnIlFC-Up9Ekrwz_luN...
1,2b8aa521-d780-4dad-989d-4c859524d5ef,RANKED_SOLO_5x5,DIAMOND,I,p_mp24OoqDibzV2hezuoTwizdGwdV0rM2xqI7z1i1D71Uh8,Justiça Marielle,24,86,92,False,False,False,False,30fKnRdigMOQV9cDG1cGLlZTv3p9NR8-zer_zwmh_k6g5z...
2,d40f0f75-79b1-48c7-96e2-318f4b05ac51,RANKED_SOLO_5x5,DIAMOND,I,eKtwgSNFv5VV8CirOIQ7Z3VJwrACEfN_gShCN-qbN9ZKK4M,twitch bojjilol,64,17,7,False,False,True,False,deXhF4r7n_bG7IuzNRQ0am0EuVjTphRvtLvmFIAKHs57x1...
3,29dd4465-d647-4db3-ab0d-e8075e5d74c1,RANKED_SOLO_5x5,DIAMOND,I,vgsW11s5ddHzIMhimj5NFMljV4fxV04hrVX-KXfcXpp3UUw,Lotrix,30,18,19,False,False,False,False,XDiwKn8ZJCIdHG61K5dLX24FyBepM8AYzD-OayfWi0Dc9o...
4,20178d72-5c23-46f0-a316-eca403b2a7e4,RANKED_SOLO_5x5,DIAMOND,I,k1JWtZWV8JoeXS4g-9CHwcb0O6aMhVemxKb9Ua1qVWDgsfY,iyu,50,75,68,False,False,False,False,kkSQlAE1Y4nVrN0WGiyvC89at9l_q6lkiKj5NHQFL8tC0y...


In [14]:

# DataFrame maior (exemplo)
df_maior = pd.DataFrame({
    'grupo': ['A', 'A', 'B', 'B'],
    'categoria': ['x', 'y', 'x', 'y'],
    'valor': [1, 2, 3, 4]
})

# DataFrame com as linhas que você deseja transformar em colunas
df_pivot = pd.DataFrame({
    'grupo': ['A', 'B', 'A', 'B'],
    'linha': ['linha1', 'linha2', 'linha3', 'linha4'],
    'valor': [10, 20, 30, 40]
})
# Pivotar o DataFrame df_pivot
df_pivot_pivotado = df_pivot.pivot(index='grupo', columns='linha', values='valor')

# Converter cada linha do DataFrame pivotado em um dicionário
dict_pivotado = df_pivot_pivotado.to_dict(orient='records')[0]

# Criar um novo DataFrame com uma única linha
df_pivot_pivotado_transposto = pd.DataFrame(dict_pivotado, index=[0])

In [15]:
# Inicialize um dataframe vazio
dfListNamesByElo_top5 = pd.DataFrame()

# Iterar sobre cada divisão
for division in listDivision:
    # Iterar sobre cada tier
    for tier in listTier:
        # Obter os dados para o tier e division atual
        data = getListNamesByElo(tier, division)
        # Criar um dataframe a partir dos dados
        df = pd.DataFrame(data)
        # Concatenar o dataframe atual ao dataframe principal
        df = df.head(5)
        dfListNamesByElo_top5 = pd.concat([dfListNamesByElo_top5, df], ignore_index=True)



# Agora dfListNamesByElo contém os dados de todas as iterações
for index, row in dfListNamesByElo_top5.iterrows():
    dfListNamesByElo_top5.loc[index, 'puuid'] = obter_id_invocador(row['summonerName'])

In [22]:
# Inicialize uma lista para armazenar os dataframes a serem concatenados
dfs_to_concat = []

for index, row in dfListNamesByElo_top5.iterrows():
    # Obtendo o nome do invocador da linha atual
    puuid = row['puuid']
    tier = row['tier']
    division = row['rank']
    summonerName = row['summonerName']
    dfUserChallenge = getChallengeById(puuid)

    df = pd.DataFrame(dfUserChallenge['challenges'])
    df['levelF'] = df['level'].astype('category').cat.codes

    dfConfig = pd.DataFrame(getConfigChallenge())
    dfConfig['name'] = dfConfig['localizedNames'].apply(lambda x: x.get('name', ''))
    dfConfig['description'] = dfConfig['localizedNames'].apply(lambda x: x.get('description', ''))
    dfConfig['shortDescription'] = dfConfig['localizedNames'].apply(lambda x: x.get('shortDescription', ''))
    dfConfig['puuid'] = puuid

    # Fundir os DataFrames df e dfConfig com base na correspondência entre df['challengeId'] e dfConfig['id']
    df_merged = pd.merge(df, dfConfig[['id', 'name','shortDescription','puuid']], left_on='challengeId', right_on='id', how='left')
    df_merged = df_merged.drop(columns=['id'])

    df_pivoted = df_merged.set_index('name')
    df_pivoted = df_pivoted.T
    dfFormated = df_pivoted.loc[['levelF']]
    # Crie um novo DataFrame com os dados atuais
    df2 = dfFormated.assign(summonerName=summonerName, puuid=puuid, tier=tier, division=division)
    # Adicione o DataFrame atual à lista de DataFrames a serem concatenados
    dfs_to_concat.append(df2)


# Concatene todos os DataFrames na lista
df_concat = pd.concat(dfs_to_concat)

# Resetar os índices do DataFrame resultante
df_concat.reset_index(drop=True, inplace=True)

In [23]:
df_concat

name,CRISTAL,IMAGINAÇÃO,ESPECIALIZAÇÃO,VIVÊNCIA,TRABALHO EM EQUIPE,COLEÇÃO,Artimanha,Festinha no Nível 1,"Farme Campeões, Não Acampamentos",Presença Devastadora,...,5 sob 5 palmos,Só no Wuju,DEZimador,A Selva é Minha,Tô Ajudando,Desrespeito,Sai pra lá!,Tá Pronto o Sorvetinho,(Sons Desumanos Estridentes),Pico de Desempenho
0,3,3,3,8,3,8,8,2,0,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8,8,8,8,8,8,2,4,3,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,9,8,3,3,3,8,3,9,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,3,2,2,7,6,2,6,3,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8,2,8,8,8,8,2,6,8,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,7,7,7,7,7,7,6,2,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
136,8,9,8,8,8,3,8,2,0,8,...,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN,NaN,NaN
137,8,9,8,8,3,8,8,9,5,8,...,NaN,NaN,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN
138,7,7,7,7,7,7,6,6,7,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
df_concat = df_concat.fillna("WOOD")
caminho_arquivo_csv = r'C:\Users\programador4\Documents\Python\R\arquivo.csv'
df_concat.to_csv(caminho_arquivo_csv, index=False, sep=',')  # index=False para não incluir o índice do DataFrame no arquivo CSV


In [19]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Seu DataFrame (suponhamos que seja df)
# df = ...

# Calcular a matriz de correlação
correlation_matrix = df.corr()

# Plotar o gráfico de correlação usando Seaborn
plt.figure(figsize=(10, 8))
sns.heatmap(df_concat, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Correlação entre as colunas do DataFrame')
plt.show()


ValueError: could not convert string to float: 'SILVER'

In [ ]:
dfFormated

In [ ]:
dfListNamesByElo_top5

In [ ]:
df_concat

In [ ]:
df_pivoted

In [ ]:
# Pivotar o DataFrame
df_pivoted = df_merged.set_index('name')
df_pivoted = df_pivoted.T
df_pivoted

In [ ]:
# Pivotar o DataFrame
df_pivoted = df_merged.set_index('name')
df_pivoted = df_pivoted.T

dfFormated = df_pivoted.loc[['level']]
dfFormated

In [ ]:
dfFormated

In [ ]:
df_merged